# Get SPACE TRACK data

In [1]:
%load_ext lab_black

In [30]:
import pandas as pd
import geopandas as gpd
import altair as alt
from datetime import timedelta
import numpy as np
import glob
import json

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### API config

In [4]:
# username = 'mstiles@grid.news'
# password = 'thespacerace123'
# output = 'json'

In [5]:
!curl -c 'params/cookies.txt' -b 'params/cookies.txt' 'https://www.space-track.org/ajaxauth/login' -d 'identity=mstiles@grid.news&password=thespacerace123'

""

### Boxscore

In [6]:
!curl --limit-rate 100K --cookie 'params/cookies.txt' 'https://www.space-track.org/basicspacedata/query/class/boxscore' -o 'data/raw/boxscore.json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35078  100 35078    0     0  36171      0 --:--:-- --:--:-- --:--:-- 36350


In [7]:
boxscore = pd.read_json("data/raw/boxscore.json")

In [8]:
boxscore.columns = boxscore.columns.str.lower()

In [9]:
boxscore = boxscore[boxscore["country"] != "ALL"].sort_values(
    "decayed_total_count", ascending=False
)

In [10]:
boxscore.country = (
    boxscore.country.str.title()
    .str.replace("Of", "of", regex=False)
    .str.replace("Peoples", "People's", regex=False)
)

In [11]:
boxscore.iloc[1]

country                      United States of America
spadoc_cd                                          US
orbital_tba                                        16
orbital_payload_count                            4012
orbital_rocket_body_count                         725
orbital_debris_count                             4477
orbital_total_count                              9230
decayed_payload_count                            1370
decayed_rocket_body_count                         726
decayed_debris_count                             4757
decayed_total_count                              6853
country_total                                   16083
Name: 106, dtype: object

In [13]:
boxscore_slim = boxscore[
    [
        "country",
        "spadoc_cd",
        "orbital_tba",
        "orbital_payload_count",
        "orbital_rocket_body_count",
        "orbital_debris_count",
        "orbital_total_count",
    ]
].sort_values("orbital_debris_count", ascending=False)

boxscore_slim.head(20)

,country,spadoc_cd,orbital_tba,orbital_payload_count,orbital_rocket_body_count,orbital_debris_count,orbital_total_count
18,Commonwealth of Independent States,CIS,5,1552,1048,6043,8648
106,United States of America,US,16,4012,725,4477,9230
77,People's Republic of China,PRC,82,517,195,3656,4450
32,France,FR,0,80,165,355,600
40,India,IND,0,103,40,75,218
49,Japan,JPN,6,205,62,54,327
25,European Space Agency,ESA,0,96,7,53,156
16,People's Republic of China/Brazil,CHBZ,0,4,0,37,41
72,Orbital Telecommunications Satellite (Globalstar),ORB,0,58,0,17,75
28,European Organization For The Exploitation of Meteorological Satellites,EUME,0,9,0,8,17


In [15]:
boxscore["orbital_total_count"].sum()

25515

In [16]:
countries = boxscore[["country", "spadoc_cd"]]

In [17]:
countries.head()

,country,spadoc_cd
18,Commonwealth of Independent States,CIS
106,United States of America,US
77,People's Republic of China,PRC
32,France,FR
40,India,IND


### Everything? 

In [18]:
!curl --limit-rate 100K --cookie 'params/cookies.txt'  'https://www.space-track.org/basicspacedata/query/class/satcat/format/json'  -O 'data/raw/all.json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23.0M  100 23.0M    0     0   100k      0  0:03:55  0:03:55 --:--:--  102k0:03:55  0:00:42  0:03:13   99k    0  0:03:55  0:02:00  0:01:55   99k   0     0   100k      0  0:03:55  0:03:15  0:00:40  100k
curl: (6) Could not resolve host: data


In [36]:
with open("data/raw/data.json", "r") as datafile:
    data = json.load(datafile)
all_df = pd.DataFrame(data)

In [37]:
all_df.head()

,INTLDES,NORAD_CAT_ID,OBJECT_TYPE,SATNAME,COUNTRY,LAUNCH,SITE,DECAY,PERIOD,INCLINATION,APOGEE,PERIGEE,COMMENT,COMMENTCODE,RCSVALUE,RCS_SIZE,FILE,LAUNCH_YEAR,LAUNCH_NUM,LAUNCH_PIECE,CURRENT,OBJECT_NAME,OBJECT_ID,OBJECT_NUMBER
0,1957-001A,1,ROCKET BODY,SL-1 R/B,CIS,1957-10-04,TTMTR,1957-12-01,96.19,65.10,938,214,None,4,0,LARGE,1,1957,1,A,Y,SL-1 R/B,1957-001A,1
1,1958-001A,4,PAYLOAD,EXPLORER 1,US,1958-02-01,AFETR,1970-03-31,88.48,33.15,215,183,None,None,0,None,1,1958,1,A,Y,EXPLORER 1,1958-001A,4
2,1958-003A,6,PAYLOAD,EXPLORER 3,US,1958-03-26,AFETR,1958-06-28,103.60,33.50,1739,117,None,5,0,None,1,1958,3,A,Y,EXPLORER 3,1958-003A,6
3,1958-004A,7,ROCKET BODY,SL-1 R/B,CIS,1958-05-15,TTMTR,1958-12-03,102.74,65.14,1571,206,None,None,0,None,1,1958,4,A,Y,SL-1 R/B,1958-004A,7
4,1958-004B,8,PAYLOAD,SPUTNIK 3,CIS,1958-05-15,TTMTR,1960-04-06,88.43,65.06,255,139,None,None,0,LARGE,1,1958,4,B,Y,SPUTNIK 3,1958-004B,8


In [38]:
all_df.columns = all_df.columns.str.lower()

In [40]:
len(all_df)

51584

In [41]:
all_df["norad_cat_id"] = all_df["norad_cat_id"].astype(str)

In [44]:
all_df.object_type.value_counts()

DEBRIS         33037
PAYLOAD        11974
ROCKET BODY     6305
TBA              268
Name: object_type, dtype: int64

In [45]:
all_df.rcs_size.value_counts()

SMALL     22351
LARGE     11731
MEDIUM     7430
Name: rcs_size, dtype: int64

In [46]:
active = all_df[all_df["decay"].isnull()]

In [47]:
len(active)

25515

In [70]:
active[active["norad_cat_id"].str.contains("12406")]

In [71]:
active.to_csv("data/processed/space_track_active.csv", index=False)

In [49]:
active_grouped = (
    active.groupby(["launch_year", "object_type"])
    .agg({"norad_cat_id": "launch_year"})
    .reset_index()
).rename(columns={"norad_cat_id": "count"})

In [56]:
alt.Chart(active_grouped).mark_bar().encode(
    x="launch_year", y="count", color="object_type"
).properties(width=650, height=350)

alt.Chart(...)

In [57]:
active_grouped_cumsum = (
    active_grouped.groupby(["launch_year", "object_type"])
    .sum()
    .groupby("object_type")
    .cumsum()
    .reset_index()
    .rename(columns={"count": "cumsum"})
)

In [58]:
active_grouped_cumsum

In [61]:
alt.Chart(active_grouped_cumsum).mark_bar().encode(
    x="launch_year", y="cumsum", color="object_type"
).properties(width=650, height=350)

alt.Chart(...)

In [ ]:
alt.Chart(active_grouped_cumsum).mark_line().encode(
    x="launch_year:O",
    y="cumsum",
    color="object_type",
).properties(width=500, height=300)

In [ ]:
active_grouped_cumsum.to_csv(
    "data/processed/active_debris_categories_cumsum.csv", index=False
)

In [ ]:
alt.Chart(active_grouped).mark_bar().encode(
    x="launch_year:O",
    y="count",
    color=alt.Color("object_type"),
).properties(width=800, height=500)

In [65]:
len(active[(active["launch_year"] == "1999") & (active["object_type"] == "DEBRIS")])

3073

In [69]:
active[
    (active["launch_year"] == "1999")
    & (active["object_type"] == "DEBRIS")
]

,intldes,norad_cat_id,object_type,satname,country,launch,site,decay,period,inclination,apogee,perigee,comment,commentcode,rcsvalue,rcs_size,file,launch_year,launch_num,launch_piece,current,object_name,object_id,object_number
10172,1999-028C,25746,DEBRIS,USA 144 DEB,US,1999-05-22,AFWTR,None,None,None,None,None,NO ELEMENTS AVAILABLE,None,0,None,1,1999,28,C,Y,USA 144 DEB,1999-028C,25746
10173,1999-028D,25747,DEBRIS,USA 144 DEB,US,1999-05-22,AFWTR,None,None,None,None,None,NO ELEMENTS AVAILABLE,None,0,None,1,1999,28,D,Y,USA 144 DEB,1999-028D,25747
10174,1999-028E,25748,DEBRIS,USA 144 DEB,US,1999-05-22,AFWTR,None,None,None,None,None,NO ELEMENTS AVAILABLE,None,0,None,1,1999,28,E,Y,USA 144 DEB,1999-028E,25748
10175,1999-028F,25749,DEBRIS,USA 144 DEB,US,1999-05-22,AFWTR,None,None,None,None,None,NO ELEMENTS AVAILABLE,None,0,None,1,1999,28,F,Y,USA 144 DEB,1999-028F,25749
10176,1999-028G,25751,DEBRIS,USA 144 DEB,US,1999-05-22,AFWTR,None,None,None,None,None,NO ELEMENTS AVAILABLE,None,0,None,1,1999,28,G,Y,USA 144 DEB,1999-028G,25751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49692,1999-025FBN,47036,DEBRIS,FENGYUN 1C DEB,PRC,1999-05-10,TSC,None,101.84,98.91,896,795,None,None,0,SMALL,7969,1999,25,FBN,Y,FENGYUN 1C DEB,1999-025FBN,47036
49693,1999-025FBT,47041,DEBRIS,FENGYUN 1C DEB,PRC,1999-05-10,TSC,None,98.78,98.21,725,676,None,None,0,SMALL,7969,1999,25,FBT,Y,FENGYUN 1C DEB,1999-025FBT,47041
50023,1999-025FCF,48517,DEBRIS,FENGYUN 1C DEB,PRC,1999-05-10,TSC,None,99.93,98.98,769,741,None,None,0,SMALL,7969,1999,25,FCF,Y,FENGYUN 1C DEB,1999-025FCF,48517
50026,1999-057TC,48526,DEBRIS,CZ-4B DEB,PRC,1999-10-14,TSC,None,95.14,98.19,538,513,None,None,0,SMALL,7969,1999,57,TC,Y,CZ-4B DEB,1999-057TC,48526
